# get 20k data 

In [ ]:
# import pandas as pd
# import os
# import subprocess
# from gradio_client import Client

In [ ]:
# Specify input and output file paths
input_file = "CVIPProjectData1.0.xlsx"  # Replace with the path to your input file
businesses_file = "businesses.txt"  # Path to save unique business names
output_file = "20kData.xlsx"  # Replace with the desired output file path
products_file = "products.xlsx"  # Path to save businesses and products
classified_file = "classified_businesses.xlsx"

In [ ]:
def extract_first_20k_rows(input_file, output_file):
    """
    Reads the input Excel file, extracts the first 2,000 rows, and writes them to a new Excel file.

    Parameters:
    input_file (str): Path to the input Excel file.
    output_file (str): Path to save the output Excel file with the extracted rows.
    """
    try:
        # Read the Excel file
        data = pd.read_excel(input_file)

        # Extract the first 2,000 rows
        first_2k = data.head(20000)

        # Save the extracted rows to a new Excel file
        first_2k.to_excel(output_file, index=False)

        print(f"Successfully extracted the first 20,000 rows to {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
extract_first_20k_rows(input_file, output_file)

# clean products data

In [ ]:
import pandas as pd

# Open the file for reading
with open('products2.txt', 'r') as file:
    lines = file.readlines()

# Process each line
cleaned_data = []
for line in lines:
    parts = line.strip().split(',')

    # Split into business name and products
    business_name = ','.join(parts[:-4])  # All but last 4 are business names
    products = [product.strip().strip('*') for product in parts[-4:]]

    # Add cleaned row to the data list
    cleaned_data.append([business_name] + products)

# Create DataFrame and export to Excel
df = pd.DataFrame(cleaned_data, columns=['Business Name', 'Product 1', 'Product 2', 'Product 3', 'Product 4'])
df.to_excel('cleaned_data.xlsx', index=False)

print("Cleaning completed. Check 'cleaned_data.xlsx'.")

# combine excel files into one

In [ ]:
import pandas as pd

# Load the specific sheet "Buyer Name" from file_1
file_1 = pd.read_excel("CVIPProjectData2.0.xlsx", sheet_name="Buyer Name")
file_2 = pd.read_excel("CVIPProjectData1.0.xlsx")
file_3 = pd.read_excel("cleaned_data.xlsx")

# Debugging: Print the shape of each file to confirm data is loaded
print(f"File 1 (Buyer Name) Shape: {file_1.shape}")
print(f"File 2 Shape: {file_2.shape}")
print(f"File 3 Shape: {file_3.shape}")

# Remove the 'BUSINESS_NAME' column from file_2 (if it exists)
file_2 = file_2.drop(columns=['BUSINESS_NAME'], errors='ignore')

# Take the first 20,000 rows of each file (if available)
file_1_trimmed = file_1.iloc[:20000]
file_2_trimmed = file_2.iloc[:20000]
file_3_trimmed = file_3.iloc[:20000]

# Debugging: Print the shape after trimming
print(f"Trimmed File 1 Shape: {file_1_trimmed.shape}")
print(f"Trimmed File 2 Shape: {file_2_trimmed.shape}")
print(f"Trimmed File 3 Shape: {file_3_trimmed.shape}")

# Concatenate all three files side by side (along columns)
combined_data = pd.concat([file_1_trimmed, file_2_trimmed, file_3_trimmed], axis=1)

# Debugging: Print the shape of the combined data
print(f"Combined Data Shape: {combined_data.shape}")

# Save the combined data into a new Excel file
output_file = "finalData.xlsx"
combined_data.to_excel(output_file, index=False)

print(f"All data combined and saved to {output_file}")


# Addding quantity, date, rate, tax,price with tax columns

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Load your existing CSV file with a different encoding
df = pd.read_csv('finalData(Sheet1).csv', encoding='ISO-8859-1')  # or use encoding='latin1'

# Generate Date (6 years of data)
start_date = datetime(2018, 1, 1)  # Start date
end_date = datetime(2024, 1, 1)    # End date
date_range = [start_date + timedelta(days=random.randint(0, (end_date - start_date).days)) for _ in range(len(df))]
df['Date'] = date_range

# Generate Quantity (random values, for each product column)
for i in range(1, 5):  # Assuming you have 4 product columns (Product 1 to Product 4)
    df[f'Quantity_Product_{i}'] = [random.randint(1, 100) for _ in range(len(df))]

# Generate Rate (random values, and round to integer)
for i in range(1, 5):  # Assuming you have 4 product columns (Product 1 to Product 4)
    df[f'Rate_Product_{i}'] = [random.randint(10, 100) for _ in range(len(df))]  # Change to integer

# Generate Tax (random values, for example between 5% and 20%)
for i in range(1, 5):  # Assuming you have 4 product columns (Product 1 to Product 4)
    df[f'Tax_Product_{i}'] = [random.uniform(0.05, 0.2) for _ in range(len(df))]

# Calculate Product Price with Tax (round to integer)
for i in range(1, 5):  # Assuming you have 4 product columns (Product 1 to Product 4)
    df[f'Product_{i}_Price_with_Tax'] = (df[f'Rate_Product_{i}'] * (1 + df[f'Tax_Product_{i}'])).round().astype(int)

# Save the updated dataframe back to a CSV
df.to_csv('DataWithProductsUpdates.csv', index=False)


In [ ]:

# Load the newly created CSV file
df_new = pd.read_csv('DataWithProductsUpdates.csv')

# Display information about the new CSV file
csv_info = df_new.info()

# Display a random sample of 20 records from the new CSV
random_records = df_new.sample(20)

(csv_info, random_records)


In [ ]:
import pandas as pd

# Read the CSV file you created earlier
df = pd.read_csv('DataWithProductsUpdates.csv')

# Save the dataframe as an Excel file
df.to_excel('DataWithProductsUpdates.xlsx', index=False)


# generating reports

In [ ]:
pip install pandas reportlab


In [ ]:
import pandas as pd
import numpy as np

# Load the sample CSV data into a DataFrame (assuming the CSV content is saved in a file named 'DataWithProductsUpdates.csv')
file_path = 'DataWithProductsUpdates.csv'
data = pd.read_csv(file_path)

# Function to calculate Amount Excluding Taxes, Amount Including Taxes, and Sales Tax @ 18%
def calculate_financials(row, quantity_col, rate_col):
    amount_excluding_taxes = row[quantity_col] * row[rate_col]
    # Set tax rate as 18%
    sales_tax = amount_excluding_taxes * 18 / 100
    amount_including_taxes = amount_excluding_taxes + sales_tax
    return amount_excluding_taxes, sales_tax, amount_including_taxes

# Add new columns for each product (Amount Excluding Taxes, Sales Tax @ 18%, Amount Including Taxes)
for i in range(1, 5):  # Assuming up to 4 products as per sample
    qty_col = f'Quantity_Product_{i}'
    rate_col = f'Rate_Product_{i}'
    excl_col = f'Amount Excluding Taxes_Product_{i}'
    tax_col = f'Sales Tax @ 18%_Product_{i}'
    incl_col = f'Amount Including Taxes_Product_{i}'
    
    # Calculate the financials and assign to new columns
    data[[excl_col, tax_col, incl_col]] = data.apply(
        lambda row: calculate_financials(row, qty_col, rate_col),
        axis=1, 
        result_type='expand'
    )

# Calculate the totals for the entire row
data['Total Amount Excluding Taxes'] = data[[f'Amount Excluding Taxes_Product_{i}' for i in range(1, 5)]].sum(axis=1)
data['Total Sales Tax @ 18%'] = data[[f'Sales Tax @ 18%_Product_{i}' for i in range(1, 5)]].sum(axis=1)
data['Total Amount Including Taxes'] = data[[f'Amount Including Taxes_Product_{i}' for i in range(1, 5)]].sum(axis=1)

# Save the updated DataFrame to a new CSV file
updated_csv_file_path = 'DataWithProductsUpdates_Updated.csv'
data.to_csv(updated_csv_file_path, index=False)

# Save the updated DataFrame to a new Excel file
updated_excel_file_path = 'DataWithProductsUpdates_Updated.xlsx'
data.to_excel(updated_excel_file_path, index=False)

updated_csv_file_path, updated_excel_file_path


# generating format 1

In [34]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, HRFlowable, Table, TableStyle, Image
import os
import random
from datetime import datetime
import requests

# Read the data
data = pd.read_csv("DataWithProductsUpdates_Updated.csv").head(30)

# Folder to save PDFs
output_folder = "pdfs/format1"
os.makedirs(output_folder, exist_ok=True)

# List of fonts to randomly select from for the logo
available_fonts = ["Helvetica-Bold", "Times-Roman", "Courier", "Helvetica"]

def generate_receipt(data_row, filename):
    # Create a PDF document
    file_path = os.path.join(output_folder, filename)
    doc = SimpleDocTemplate(file_path, pagesize=letter, showBoundary=True)
    
    # Create styles for the PDF
    styles = getSampleStyleSheet()
    style_subtitle = styles['Normal']
    style_business_name = ParagraphStyle(name="BusinessName", fontSize=14, alignment=1, fontName="Helvetica-Bold")
    
    # Create the PDF content
    content = []
    
    # Add logo image
    logo_url = "https://assets.streamlinehq.com/image/private/w_240,h_240,ar_1/f_auto/v1/icons/cashier/receipt-qnzwazerc6qds88389goz.png/receipt-aul51ose8jdkljvhudgdt.png?_a=DAJFJtWIZAAC"
    logo_image_path = "logo.png"
    response = requests.get(logo_url)
    with open(logo_image_path, 'wb') as f:
        f.write(response.content)
    logo = Image(logo_image_path, width=50, height=50)
    content.append(logo)
    content.append(Spacer(1, 10))
    
    # Add business name below the logo
    business_name = f"<b>{data_row['Business Name']}</b>"
    content.append(Paragraph(business_name, style_business_name))
    content.append(Spacer(1, 25))

    # Horizontal rule
    content.append(HRFlowable(width="100%", thickness=1, color=colors.black))
    content.append(Spacer(1, 10))

    # Supplier and Customer Info
    supplier_info = f"""
    <b>Supplier:</b><br />
    NTN: {data_row['NTN']}<br />
    Name: {data_row['NAME']}<br />
    S.T.Reg. No: {data_row['S.T.Reg. No']}<br />
    Address: {data_row['Address']}<br />
    """

    customer_info = f"""
    <b>Customer:</b><br />
    NTN: {data_row['Customer_NTN']}<br />
    Name: {data_row['Customer_NAME']}<br />
    S.T.Reg. No: {data_row['Customer_S.T.Reg. No']}<br />
    Address: {data_row['Customer_Address']}<br />
    Phone: {data_row['Customer_Phone Number']}<br />
    """
    table_data = [[Paragraph(supplier_info, style_subtitle), Paragraph(customer_info, style_subtitle)]]
    table = Table(table_data, colWidths=[200, 200])
    table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (0, -1), 'TOP'),
        ('FONTNAME', (0, 0), (-1, -1), "Helvetica"),
        ('GRID', (0, 0), (-1, -1), 0, colors.black),
    ]))
    content.append(table)
    content.append(Spacer(1, 20))

    # Product Table
    data_for_table = [
        ["Product Name", "Quantity", "Rate", "$ Excl. Tax", "Sales Tax @18%", "$ Incl. Tax"]
    ]

    # Convert all values to strings, replacing NaN or None with an empty string
    product_names = [str(data_row.get(f"Product {i}", "")) for i in range(1, 5)]
    quantities = [str(data_row.get(f"Quantity_Product_{i}", 0)) for i in range(1, 5)]
    rates = [data_row.get(f"Rate_Product_{i}", 0.0) for i in range(1, 5)]
    excl_tax = [data_row.get(f"Amount Excluding Taxes_Product_{i}", 0.0) for i in range(1, 5)]
    sales_tax = [data_row.get(f"Sales Tax @ 18%_Product_{i}", 0.0) for i in range(1, 5)]
    incl_tax = [data_row.get(f"Amount Including Taxes_Product_{i}", 0.0) for i in range(1, 5)]

    # Populate the table rows
    for i in range(4):
        data_for_table.append([
            Paragraph(product_names[i], style_subtitle), 
            Paragraph(str(quantities[i]), style_subtitle), 
            Paragraph(f"{rates[i]:.2f}", style_subtitle), 
            Paragraph(f"{excl_tax[i]:.2f}", style_subtitle),
            Paragraph(f"{sales_tax[i]:.2f}", style_subtitle),
            Paragraph(f"{incl_tax[i]:.2f}", style_subtitle)
        ])

    table = Table(data_for_table, colWidths=[100, 60, 50, 80, 90, 80])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 1), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), "Helvetica-Bold"),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('VALIGN', (0, 0), (-1, -1), 'TOP')
    ]))
    content.append(table)
    content.append(Spacer(1, 20))

    # Total Price Table
    total_price = data_row['Total Amount Including Taxes']
    total_table_data = [
        ["Total Amount Excl. Taxes:", f"{data_row['Total Amount Excluding Taxes']:.2f}"],
        ["Total Sales Tax @ 18%:", f"{data_row['Total Sales Tax @ 18%']:.2f}"],
        ["Total Amount Incl. Taxes:", f"{total_price:.2f}"]
    ]
    total_table = Table(total_table_data, colWidths=[245, 190])
    total_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, -1), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, -1), "Helvetica-Bold"),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 10),
        ('TOPPADDING', (0, 0), (-1, -1), 10),
    ]))
    content.append(total_table)
    content.append(Spacer(1, 20))

    # Date
    current_date = datetime.now().strftime("%Y-%m-%d")
    date_paragraph = f"<b>Date:</b> {current_date}"
    content.append(Paragraph(date_paragraph, style_subtitle))
    content.append(HRFlowable(width="100%", thickness=1, color=colors.black))

    # Build the document
    doc.build(content)

# Generate receipt for each row in the first 5 records
for index, row in data.iterrows():
    filename = f"receipt_{index + 1}.pdf"
    generate_receipt(row, filename)

print(f"PDFs generated and saved in the folder '{output_folder}'")


/var/folders/f4/8cgd8yrj4nn28wpj00ct40040000gn/T/ipykernel_3068/1519302739.py:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("DataWithProductsUpdates_Updated.csv").head(30)


PDFs generated and saved in the folder 'pdfs/format1'


In [ ]:
# pip install pdf2image pillow

In [35]:
import os
from pdf2image import convert_from_path

# Input and output folder paths
input_folder = "pdfs/format1"
output_folder = "picOutputs/format1"
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist

def convert_pdf_to_jpg(pdf_folder, output_folder):
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            # Path to the input PDF file
            pdf_path = os.path.join(pdf_folder, pdf_file)
            
            # Convert PDF to images (JPG)
            images = convert_from_path(pdf_path, dpi=300)  # Default format is JPG
            
            # Save each page as JPG file
            for i, image in enumerate(images):
                jpg_filename = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}.jpg")
                image.save(jpg_filename, format="JPEG")
                print(f"Saved: {jpg_filename}")

# Run the conversion
convert_pdf_to_jpg(input_folder, output_folder)

print(f"JPG files saved in '{output_folder}'")


Saved: picOutputs/format1/receipt_21.jpg
Saved: picOutputs/format1/receipt_20.jpg
Saved: picOutputs/format1/receipt_22.jpg
Saved: picOutputs/format1/receipt_23.jpg
Saved: picOutputs/format1/receipt_27.jpg
Saved: picOutputs/format1/receipt_26.jpg
Saved: picOutputs/format1/receipt_18.jpg
Saved: picOutputs/format1/receipt_24.jpg
Saved: picOutputs/format1/receipt_30.jpg
Saved: picOutputs/format1/receipt_25.jpg
Saved: picOutputs/format1/receipt_19.jpg
Saved: picOutputs/format1/receipt_9.jpg
Saved: picOutputs/format1/receipt_8.jpg
Saved: picOutputs/format1/receipt_3.jpg
Saved: picOutputs/format1/receipt_2.jpg
Saved: picOutputs/format1/receipt_1.jpg
Saved: picOutputs/format1/receipt_5.jpg
Saved: picOutputs/format1/receipt_4.jpg
Saved: picOutputs/format1/receipt_6.jpg
Saved: picOutputs/format1/receipt_7.jpg
Saved: picOutputs/format1/receipt_14.jpg
Saved: picOutputs/format1/receipt_28.jpg
Saved: picOutputs/format1/receipt_29.jpg
Saved: picOutputs/format1/receipt_15.jpg
Saved: picOutputs/format1

In [1]:
pip install pytesseract pillow opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_12_0_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_12_0_x86_64.whl (56.5 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/local/bin/tesseract'



In [6]:
from PIL import Image


In [12]:
import cv2
from PIL import Image
import pytesseract

# Configure Tesseract command
pytesseract.pytesseract.tesseract_cmd = '/usr/local/bin/tesseract'

# Load the image
image_path = "receipt_1.jpg"
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding
_, binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Save and load the processed image for OCR
cv2.imwrite("processed_receipt.jpg", binary_image)
processed_image = Image.open("processed_receipt.jpg")

# Perform OCR
text = pytesseract.image_to_string(processed_image)
print("Extracted Text:")
print(text)


[ WARN:0@0.021] global loadsave.cpp:241 findDecoder imread_('receipt_1.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [13]:
pip install yolov5


INFO: pip is looking at multiple versions of yolov5 to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of yolov5 to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
Requested yolov5 from https://files.pythonhosted.org/packages/96/b2/a655ed5772f8990705980d0d611722f3cc4e5482b18719c9755ae8da6de0/yolov5-5.0.10-py36.py37.py38-none-any.whl has invalid metadata: Expected matching RIGHT_PARENTHESIS for LEFT_PARENTHESIS, after version specifier
    pycocotools (>=2.0s)
                ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested yolov5 from https://files.pythonhosted.org/packages/5f/07/42a5dffd0fe836bf0ebea

In [33]:
import torch
from PIL import Image
import cv2

# Load a pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # 'yolov5s' is a small model for fast inference

# Load image
image_path = 'receipt_1.jpg'
img = Image.open(image_path)

# Perform detection
results = model(img)

# Display the results
results.show()  # Display detected objects

# You can also print results
results.print()  # Show detected objects with their confidence


ModuleNotFoundError: No module named 'torch'

In [30]:
pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

SyntaxError: invalid syntax (3361530279.py, line 1)

In [32]:
!pip3 install pytorch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pytorch (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
      Traceback (most recent call last):
        File "/Users/apple/Desktop/semester 7/cvip semester project/env/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
          ~~~~^^
        File "/Users/apple/Desktop/semester 7/cvip semester project/env/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/apple/Desktop/semester 7/cvip semester project/env/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_pr